<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Session 2:  Data Cleaning and Formatting in Python
<a id='beginning'></a>

Having collected the data does not always allow you to produce some analytics right away. There is often a lot of pre processing to be done. 

This session is about:

* Cleaning: making sure each cell has a value that could be used in your coming procedures. The _impurities_ do not allow formatting the data correctly: commas instead of points and viceversa, blanks/spaces, unneeded symbols (dollar, euro symbols), or non-standard symbols to represent missing values.

* Fomatting: making sure the clean value is in the right data type. if you are going to do text analysis, you may need to get rid of repetitive words, normalize them into lower case, and turn them back to their root or stem. For statistical work, you need to differentiate among nominal, ordinal, numerical and strings.

Let me use the next table with information on **Democracy and its components** by country:

In [ ]:
from IPython.display import IFrame  
wikiLink="https://en.wikipedia.org/wiki/Democracy_Index#Components" 
IFrame(wikiLink, width=900, height=500)

You should have observed the following:

1. One score of democracy is offered for the participant countries.
2. Four levels of democracy are offered for the participant countries.
3. The score is computed from other variables

Let's try to get the table using _pandas_:

In [ ]:
import pandas as pd

wikiTables=pd.read_html(wikiLink, # link
                        header=0, # where is the header?
                        flavor='bs4', # helper to translate html
                        attrs={'class': 'wikitable sortable'}) # attributes to identify element(s)

Remember the object **wikiTables** is a list. I know before hand that our DF is the third one:

In [ ]:
wikiTables[2] # you should vary this index until you get the table you need!

Pandas will show you the _head_ and the _tail_ of the dataframe. There you can verify if the column names are well positioned. Let me save the data frame temporarily:

In [ ]:
DFwiki=wikiTables[2].copy()

# I. Cleaning

1. Keep columns needed

In [ ]:
# columns present
DFwiki.columns

In [ ]:
# dropping
DFwiki.drop(columns=DFwiki.columns[[0,1,5]],inplace=True)#inplace!!
DFwiki

2. Check column names

In [ ]:
DFwiki.columns.to_list()

You should avoid spaces and weird strings in columns names, what about...

In [ ]:
DFwiki.columns.str.replace("\xad|\s","",regex=True)

Now you can alter the column names:

In [ ]:
DFwiki.columns=DFwiki.columns.str.replace("\xad|\s","",regex=True)

3. Check columns with strings:

In [ ]:
DFwiki.iloc[:,:2]

In general, you need to check that they do not have leading nor trailing spaces:

In [ ]:
" Peru ".strip()

Pandas has its own strip:

In [ ]:
DFwiki.Country.str.strip()

You can use that several times, because you can not use that function on several columns directly; unless...

In [ ]:
DFwiki.iloc[:,:2].apply(lambda x: x.str.strip())

You can make the changes:

In [ ]:
DFwiki.iloc[:,:2]=DFwiki.iloc[:,:2].apply(lambda x: x.str.strip())

4. Check levels of Categorical columns

In [ ]:
DFwiki.iloc[:,1]

The column **Regimetype** is a category. You should always ask a frequency table to detect possible errors:

In [ ]:
DFwiki.Regimetype.value_counts()

Notice the similar values that only have one count. Those are not levels. If you visit the webpage you will see they are labels of sections in the table. Let me get those values as a list:

In [ ]:
tableCounts=DFwiki.Regimetype.value_counts()
tableCounts[tableCounts==1].index.to_list()

Now, let me confirm they should be erased from the data:

In [ ]:
badLevels=tableCounts[tableCounts==1].index.to_list()
DFwiki[DFwiki.Regimetype.isin(badLevels)]

Then, we are good without them:

In [ ]:
DFwiki=DFwiki[~DFwiki.Regimetype.isin(badLevels)]
DFwiki

# II. Formatting


1. Check the data types

First, see what data types have been assigned by Python to each column:

In [ ]:
DFwiki.info()

If the columns have texts, Python will say it is an **object**. That is OK for _Country_, but not for the others. _Regimetype_ is a **category**, and all the other columns are **numeric** values.

2. Text to categorical



In [ ]:
DFwiki.Regimetype.value_counts()

In the cleaning process we got rid of the wrong levels, now we need to set the right data type:

In [ ]:
from pandas.api.types import CategoricalDtype

# notice ascending order of levels
regimeLevels=["Authoritarian", "Hybrid regime","Flawed democracy", "Full democracy"]

# create data type
regimeOrdered = CategoricalDtype(categories=regimeLevels, ordered=True)

# make the change:
DFwiki['Regimetype']=DFwiki.Regimetype.astype(regimeOrdered)

See the changes:

In [ ]:
DFwiki.Regimetype

3. Numbers that need to be numerical type

This is simple with **to_numeric** from pandas. See how it works:


In [ ]:
DFwiki.Overallscore

In [ ]:
pd.to_numeric(DFwiki.Overallscore,errors='coerce')

However, that functions can not be applied to a set columns, but one column at a time.

In [ ]:
# this will not work:
# pd.to_numeric(DFwiki.iloc[:,2:])

Let me show you some strategies:

In [ ]:
# copy
DFtest1=DFwiki.copy()
# creating dictionaries:
where=DFtest1.columns[2:]
newCols={col:pd.to_numeric(DFtest1.loc[:,col],errors='coerce') for col in where}

#then:
DFtest1[where]=pd.DataFrame(newCols)
DFtest1.info()


In [ ]:
# using loops:

DFtest2=DFwiki.copy()

for col in DFtest2.columns[2:]:
    DFtest2.loc[:,col]=pd.to_numeric(DFtest2.loc[:,col],errors='coerce')

DFtest2.info()

A simpler way might be just **apply** the function:

In [ ]:
DFtest2[where]=DFtest2.loc[:,where].apply(pd.to_numeric,errors='coerce')
DFtest2.info()

Let's follow the last strategy:

In [ ]:
# Apply function
where=DFwiki.columns[2:]
DFwiki[where]=DFwiki.loc[:,where].apply(pd.to_numeric,errors='coerce')

In [ ]:
# result

DFwiki.info()

The DF now has the right data types. 

It would be a good idea to save this work:

In [ ]:
DFwiki.to_csv("demoindex.csv",index=False)
DFwiki.to_pickle("demoindex.pkl")

CSV files are very common, but let me show a disadvantage:

In [ ]:
democsv=pd.read_csv('demoindex.csv')
democsv.info()

Compare it to the pickle version:

In [ ]:
demopkl=pd.read_pickle('demoindex.pkl')
demopkl.info()